In [1]:
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
from konlpy.tag import Okt
import logging
import time
import re
import os
import json
from tqdm import tqdm
import torch
import os
import json
import unicodedata
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from pdf2image import convert_from_path
import pytesseract
# Vector stores
import fitz  # PyMuPDF
import pymupdf4llm
from pymupdf4llm import to_markdown
from PyPDF2 import PdfReader

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cuda' if torch.cuda.is_available() else device  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

from langchain_unstructured import UnstructuredLoader

def get_embedding():
    
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ")  # 잘못된 인코딩 문자 제거
    return text


mps


### markdown, table

In [16]:
def process_pdf(pdf_path): 
    md_text = pymupdf4llm.to_markdown(pdf_path)

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
        ("#####", "Header 5"),
        ("######", "Header 6")
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512, chunk_overlap=64
    )

    splits = text_splitter.split_documents(md_chunks)

    for i in splits:
        metadata = {'Source_path': pdf_path}
        i.metadata = {**i.metadata, **metadata}
    return splits


def make_db(df):
    documents = []
    
    pdf_files = df['Source_path'].unique()
    for pdf_file in tqdm(pdf_files):
        # 문서 로드
        documents.extend(process_pdf(pdf_file))
        
    print(f"Total number of documents: {len(documents)}")

    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    return faiss

In [3]:
# md_text = pymupdf4llm.to_markdown(test_df.iloc[0]['Source_path'])


# headers_to_split_on = [
#     ("#", "Header 1"),
#     ("##", "Header 2"),
#     ("###", "Header 3"),
# ]

# md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
# md_chunks = md_header_splitter.split_text(md_text)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=512, chunk_overlap=64
# )

# splits = text_splitter.split_documents(md_chunks)

# for i in splits:
#     metadata = {'Source_path': test_df.iloc[0]['Source_path']}
#     i.metadata = {**i.metadata, **metadata}
# faiss = FAISS.from_documents(splits, embedding=get_embedding())

# docs = faiss.similarity_search(
#     query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
#     filter={"Source_path": test_df.iloc[0]['Source_path']},
#     k = 99,
#     fetch_k = 20000
# )
# buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
# faiss_retriever = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k,'fetch_k': 50})
# knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
# knn_retriever.k = k
# bm_retriever = OktBM25Retriever.from_documents(docs)
# bm_retriever.k = k

# ensenble_retriever = EnsembleRetriever(retrievers=[faiss_retriever, knn_retriever,bm_retriever], weight=[0.4, 0.3, 0.3])


### konlptextsplitter, pytessearct

In [4]:
# def ocr_extract_text_from_pdf(pdf_path):
#     documents = []
#     try:
#         # PDF를 이미지로 변환
#         images = convert_from_path(pdf_path, dpi=500)
        
#         # 각 페이지에서 OCR로 텍스트 추출
#         for i, image in enumerate(images):
#             text = pytesseract.image_to_string(image, lang='kor+eng')
#             text = clean_text(normalize_string(text))
#             # 각 페이지마다 별도의 Document 생성
#             documents.append(Document(page_content=text, metadata={"source": pdf_path, "page": i + 1}))
    
#     except Exception as e:
#         print(f"Error extracting text from {pdf_path}: {e}")
    
#     return documents

# def load_pdf_file(pdf_file):
#     try:
#         # OCR을 이용한 텍스트 추출
#         documents = ocr_extract_text_from_pdf(pdf_file)
#         return documents
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []

# def split_documents_with_konlpy(documents):
#     adjusted_documents = []
#     konlpy_splitter = KonlpyTextSplitter(separator='\n\n')

#     for doc in documents:
#         # KonlpyTextSplitter로 문장 단위 분리
#         sentences = konlpy_splitter.split_text(doc.page_content)
#         for sentence in sentences:
#             adjusted_documents.append(Document(page_content=sentence, metadata=doc.metadata))
    
#     return adjusted_documents

# def make_db(df, chunk_name="train", chunk_size=800, chunk_overlap=60):
#     documents = []
#     chunk_file_path = f"{chunk_name}_chunks.json"

#     if os.path.exists(chunk_file_path):
#         print(f"Loading chunks from {chunk_file_path}...")
#         documents = load_chunks(chunk_file_path)
#     else:
#         pdf_files = df['Source_path'].unique()
#         with ThreadPoolExecutor() as executor:
#             results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
        
#         for result in results:
#             documents.extend(result)
        


#         # 청크 데이터를 저장
#         save_chunks(documents, chunk_file_path)

#     # KonlpyTextSplitter로 문장 단위로 먼저 분할
#     documents = split_documents_with_konlpy(documents)
    
#     # make_db에서 RecursiveCharacterTextSplitter로 청크를 분할
#     recursive_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         separators=["\n\n", "."]
#     )
#     documents = recursive_splitter.split_documents(documents)
    
#     faiss = FAISS.from_documents(documents, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(documents)
    
#     return faiss, bm

# def save_chunks(documents, path):
#     chunk_data = []
#     for doc in documents:
#         chunk_data.append({
#             "page_content": doc.page_content,
#             "metadata": doc.metadata
#         })
    
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(chunk_data, f, ensure_ascii=False, indent=4)
    
#     print(f"Chunks saved to {path}")

# def load_chunks(path):
#     with open(path, "r", encoding="utf-8") as f:
#         chunk_data = json.load(f)
    
#     documents = []
#     for chunk in chunk_data:
#         documents.append(Document(page_content=chunk["page_content"], metadata=chunk["metadata"]))
    
#     print(f"Chunks loaded from {path}")
#     return documents

### 언스트럭처

In [5]:
# def normalize_string(s):
#     try:
#         normalized = unicodedata.normalize('NFC', s)
#         return normalized.encode('utf-8', errors='replace').decode('utf-8')
#     except Exception:
#         return s
# def clean_text(text):
#     text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
#     text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
#     return text

# def load_pdf_file(pdf_file):
#     try:
#         pdf_loader = UnstructuredLoader(pdf_file,
#                                         chunking_strategy = 'by_title',
#                                         max_characters = 1000,
#                                         overlap = 200)
#         return pdf_loader.load()
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []

# def make_db(df, chunk_name="train"):
#     documents = []
#     chunk_file_path = f"{chunk_name}_chunks.json"

#     if os.path.exists(chunk_file_path):
#         # 이미 저장된 청크 데이터가 있으면 불러오기
#         print(f"Loading chunks from {chunk_file_path}...")
#         documents = load_chunks(chunk_file_path)
#     else:
#         # 청크 데이터가 없으면 PDF 파일을 처리하여 새로 생성
#         pdf_files = df['Source_path'].unique()

#         # tqdm으로 파일 처리 진행 상황 표시
#         with ThreadPoolExecutor() as executor:
#             results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
        
#         for result in results:
#             documents.extend(result)
        
#         # 정규화
#         for doc in documents:
#             doc.page_content = normalize_string(doc.page_content)
        
#         # 청크 데이터를 저장
#         save_chunks(documents, chunk_file_path)
    
#     # FAISS DB 만들기 (메타데이터 포함)
#     faiss = FAISS.from_documents(documents, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(documents)
    
#     return faiss, bm

# import json

# def save_chunks(documents, path):
#     chunk_data = []
#     for doc in documents:
#         chunk_data.append({
#             "page_content": doc.page_content,
#             "metadata": doc.metadata
#         })
    
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(chunk_data, f, ensure_ascii=False, indent=4)
    
#     print(f"Chunks saved to {path}")

# from langchain.docstore.document import Document

# def load_chunks(path):
#     with open(path, "r", encoding="utf-8") as f:
#         chunk_data = json.load(f)
    
#     documents = []
#     for chunk in chunk_data:
#         documents.append(Document(page_content=chunk["page_content"], metadata=chunk["metadata"]))
    
#     print(f"Chunks loaded from {path}")
#     return documents


### pdf 로더

In [6]:
# def get_embedding():
#     device = 'cuda' if torch.cuda.is_available() and 'mps' in torch.cuda.get_device_capability() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
#     embeddings = HuggingFaceEmbeddings(
#         model_name='jhgan/ko-sroberta-multitask',
#         model_kwargs={'device': device},
#         encode_kwargs={'normalize_embeddings': True})
#     return embeddings

# def normalize_string(s):
#     try:
#         normalized = unicodedata.normalize('NFC', s)
#         return normalized.encode('utf-8', errors='replace').decode('utf-8')
#     except Exception:
#         return s

# def clean_text(text):
#     text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
#     text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
#     return text

# def extract_text_and_tables(pdf_path):
#     text_content = ""
#     try:
#         # fitz를 사용한 텍스트 추출
#         doc = fitz.open(pdf_path)
#         for page in doc:
#             text = page.get_text("text")
#             if text:
#                 text = clean_text(normalize_string(text))
#                 text_content += text + "\n"

#         # pdfplumber를 사용한 표 추출
#         with pdfplumber.open(pdf_path) as pdf:
#             for page in pdf.pages:
#                 tables = page.extract_tables()
#                 if tables:
#                     text_content += "\n__table__\n"
#                     for table in tables:
#                         if table:
#                             for row in table:
#                                 processed_row = []
#                                 for cell in row:
#                                     if cell is None:
#                                         processed_row.append(" ")  # 병합된 셀 처리
#                                     else:
#                                         cell_text = normalize_string(clean_text(str(cell).strip()))
#                                         if "\n" in cell_text:
#                                             cell_text = " ".join(cell_text.split())
#                                         processed_row.append(cell_text)
#                                 row_text = " | ".join(processed_row)
#                                 text_content += row_text + "\n"
#                     text_content += "__end_table__\n"
#     except Exception as e:
#         print(f"Error extracting text and tables from {pdf_path}: {e}")
#     return text_content

# class CustomPDFLoader:
#     def __init__(self, file_path):
#         self.file_path = file_path

#     def load(self):
#         documents = []
#         try:
#             # 텍스트와 표를 모두 추출
#             text_content = extract_text_and_tables(self.file_path)

#             # 메타데이터 생성 및 Document 객체로 변환
#             metadata = {
#                 "source": self.file_path,
#             }
#             documents.append(Document(page_content=text_content, metadata=metadata))
#         except Exception as e:
#             print(f"Error loading PDF file {self.file_path}: {e}")
#         return documents

# def chunk_documents(documents):
#     adjusted_documents = []
#     for doc in documents:
#         content = doc.page_content
#         chunks = []
        
#         # 표 구분자를 기준으로 분할
#         table_sections = content.split("__table__")
        
#         for section in table_sections:
#             if "__end_table__" in section:
#                 table_content = section.split("__end_table__")[0]
#                 chunks.append(table_content)
#             else:
#                 chunk_splitter = RecursiveCharacterTextSplitter(
#                     chunk_size=1000,  # 텍스트 청크 크기를 키움
#                     chunk_overlap=200,
#                     separators=["\n\n", ".", "?", "!", "\n"]
#                 )
#                 normal_chunks = chunk_splitter.split_text(section)
#                 chunks.extend(normal_chunks)

#         for chunk in chunks:
#             adjusted_documents.append(Document(page_content=chunk, metadata=doc.metadata))
    
#     return adjusted_documents

# def format_docs(docs):
#     context = ""
#     for doc in docs:
#         context += doc.page_content
#         context += '\n\n'
#     return context

# def make_db(df):
#     documents = []
#     pdf_files = df['Source_path'].unique()

#     # tqdm으로 파일 처리 진행 상황 표시
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
    
#     for result in results:
#         documents.extend(result)
    
#     # 정규화
#     for doc in documents:
#         doc.page_content = normalize_string(doc.page_content)
    
#     # 표가 망가지지 않도록 청크 분할 처리
#     chunks = chunk_documents(documents)
#     print(f"Total number of chunks: {len(chunks)}")
    
#     # FAISS DB 만들기 (메타데이터 포함)
#     faiss = FAISS.from_documents(chunks, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(chunks)
    
#     return faiss, bm

# def load_pdf_file(pdf_file):
#     try:
#         pdf_loader = CustomPDFLoader(pdf_file)
#         return pdf_loader.load()
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []


### ..

In [17]:
# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [18]:
faiss = make_db(test_df)

100%|██████████| 9/9 [00:07<00:00,  1.15it/s]


Total number of documents: 310


In [37]:
i = 48
k = 4

def format_docs(docs):
    context = ""
    for doc in docs:
        # Header 정보를 순서대로 추가
        for header_level in range(1, 6):
            header_key = f'Header {header_level}'
            if header_key in doc.metadata:
                context += f"{header_key}: {doc.metadata[header_key]}\n"
        # 문서 내용 추가
        context += doc.page_content
        context += '\n---\n'
    return context.strip()


docs = faiss.similarity_search(
    query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
    filter={"Source_path": test_df.iloc[i]['Source_path']},
    k = 99,
    fetch_k = 20000
)
buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
faiss_retriever_mmr = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k})
faiss_retriever_sim = buff_faiss.as_retriever(search_kwargs={"k": k})
knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
knn_retriever.k = k
bm_retriever = OktBM25Retriever.from_documents(docs)
bm_retriever.k = k

ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever_mmr,faiss_retriever_sim,knn_retriever,bm_retriever], weight=[0.25 ,0.25,0.25 ,0.25])

print(test_df.iloc[i]['Question'])
print(test_df.iloc[i]['Source_path'])

# results_bm = bm_retriever.invoke(test_df.iloc[i]['Question'])
# results_faiss = faiss_retriever.invoke(test_df.iloc[i]['Question'])
# for i in results_bm:
#     print(i.metadata)
#     print('-------------------')
# print('============================================')
# for i in results_faiss:
#     print(i.metadata)
#     print('-------------------')
# print('============================================')
results = ensemble_retriever.invoke(normalize_string(test_df.iloc[i]['Question']))
for i in results:
    print(i.metadata)
    print(i.page_content)
    print('-------------------')
print(len(format_docs(results)))

핵심재정사업 성과관리제도를 안착시키기 위해 필요한 노력과 성과 정보를 학습의 도구로 활용하는 방안은 무엇인가?
./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf
{'Header 1': 'ISSUE & FOCUS', 'Header 2': '05', 'Source_path': './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'}
**정사업 성과관리제도를 안착시켜 나갈 수 있기를 기대**  
작업반 간 모범사례 등의 공유를 위한 교훈 저장소 생성 등으로 성과정보가 사업추진기관에 의해 학습의  
도구로서 유용하게 쓰이게끔 하고 성과중심의 조직문화를 형성해 가는 것이 필요  
-  성공사례와 목표 결과를 공유할 수 있는 다양하고 효과적인 커뮤니케이션 전략을 개발해 범부처 차원의  
우선순위 목표에 대한 국민적 관심과 열린 대화를 형성  
-----  
**FIS** **ISSUE & FOCUS**
-------------------
{'Header 1': 'ISSUE & FOCUS', 'Header 3': '핵심재정사업 성과관리', 'Source_path': './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'}
을 일목요연하게 정리해 보고자 한다. 또한 향후 제도를 성공적으로 안착시키기 위해 핵심재정사업 성과관리체계를 어떠한 방향  
으로 운영해 나가는 것이 필요할지 살펴보았다.
-------------------
{'Header 1': 'ISSUE & FOCUS', 'Header 2': '04', 'Source_path': './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'}
영역에 대한 구현 계획 및 조정을 강화 및 수행하기 위한 팁과 도구를 개발  
-  목표 팀 간 모범사례 등의 공유를 위한 교훈 저장소 생성(예를 들어, 목표 팀이 목표를 설정, 계획 및 실행  
하는 데 도움이 

In [21]:
format_docs(results)

'Header 1: 사  업  명\n(74) 노인장기요양보험 사업운영 (2231-303)\n---\nHeader 1: 5. 사업근거 및 추진경위\n노인장기요양보험법 제35조의2(장기요양기관 재무·회계기준) ① 장기요양기관의 장은 보건복지부령으로 정  \n하는 재무·회계에 관한 기준(이하 “장기요양기관 재무·회계기준”이라 한다)에 따라 장기요양기관을 투명  \n하게 운영하여야 한다. 다만, 장기요양기관 중 「사회복지사업법」제34조에 따라 설치한 사회복지시설은  \n같은 조 제3항에 따른 재무ㆍ회계에 관한 기준에 따른다.  \n노인장기요양보험법 제58조(국가의 부담) ① 국가는 매년 예산의 범위 안에서 해당 연도 장기요양  \n보험료 예상수입액의 100분의 20에 상당하는 금액을 공단에 지원한다.\n② 국가와 지방자치단체는 대통령령으로 정하는 바에 따라 의료급여수급권자의 장기요양급여\n비용, 의사소견서 발급비용, 방문간호지시서 발급비용 중 공단이 부담하여야 할 비용(제40조제\n1항 단서 및 제3항제1호에 따라 면제 및 감경됨으로 인하여공단이 부담하게 되는 비용을 포\n함한다) 및 관리운영비의 전액을 부담한다.\n---\nHeader 1: 7. 사업 집행절차\n○ 노인장기요양보험 운영지원  \n○ 공무원·사립학교교원 등 장기요양보험료 국가부담금  \n○ 기타의료급여 수급권자 급여비용 국가부담금  \n○ 장기요양기관 재무회계 프로그램 운영  \n부처 피출연·피보조 기관  \n보건복지부 ⇒ 한국사회보장정보원  \n(972백만원) (972백만원) (972백만원)  \n-----\n---'

In [11]:
total = []
for i in range(len(test_df)):
    query = test_df.iloc[i]['Question']
    results = test_ensemble_retriever.invoke(query)
    chunk_sizes = [len(i.page_content) for i in results]
    total.extend(chunk_sizes)

NameError: name 'test_ensemble_retriever' is not defined

In [ ]:
import numpy as np
np.mean(total)

1875.1284403669724

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def format_docs(docs):
    context = ""
    i = 1
    for doc in docs:
        context += f"Document: {i}" +"Source:"+ doc.metadata['file_path'] +'\n'
        context += doc.page_content
        context += '\n---\n'
        i += 1
    return context


i= 35

query = test_df.iloc[i]['Question']
print(query)
print(test_df.iloc[i]['Source_path'])
results = test_ensemble_retriever.invoke(normalize_string(query))
def equal_path(contexts, source_path):
    adjusted_docs = []
    for doc in contexts:
        if doc.metadata['file_path'] == source_path:
            adjusted_docs.append(doc)
    return adjusted_docs
results = equal_path(results, test_df.iloc[i]['Source_path'])
for result in  results:
    print(result.metadata)
    print(result.page_content)
    print('-------------------')


21년 국정감사에서 에너지 바우처 사업에 대한 주요 지적사항은 무엇이었나요?
./test_source/산업통상자원부_에너지바우처.pdf


KeyError: 'file_path'